### Imports

In [2]:
import numpy as np
import random
# import to do training
from tpg.trainer import Trainer
# import to run an agent (always needed)
from tpg.agent import Agent
# faster training
import multiprocessing as mp
# visual tools
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

In [3]:
def animate_path(sequence):
    current_map = np.zeros((10, 10))
    # add barrier
    for i in range(5):
        current_map[(4, i)] = 5
    # animate the run!
    for i in range(len(sequence)):
        time.sleep(0.5)
        if i == 0:
            current_map[sequence[i]] = 1
            clear_output(wait=True)
            print(0)
            print(current_map)
        else:
            current_map[sequence[i-1]] = 0
            current_map[sequence[i]] = 1
            clear_output(wait=True)
            print(i)
            print(current_map)

In [4]:
# example of above 
seq = [(0,1), (0,2), (0, 3), (1, 3), (2, 4)]
animate_path(seq)

4
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [5]:
# todo: make generalizable for any GridWorld
def heat_map(sequence):
    x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    y = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    plt.xticks(ticks=np.arange(len(x)),labels=x)
    plt.yticks(ticks=np.arange(len(y)),labels=y)
    plt.imshow(sequence, cmap='cool')

### Grid World

In [6]:
# based (for now) on Downing's Figure 4.1
class GridWorldv1:
    def __init__(self, rows, cols, win_state, starting_state):  
        self.rows = rows
        self.cols = cols
        self.starting_state = starting_state
        self.win_state = win_state
        self.current_state = self.starting_state

    def sample_action (self):
        rand = random.uniform(0, 1)
        if (rand >= 0) and (rand < 0.25):
            return 0
        elif (rand >= 0.25) and (rand < 0.5):
            return 1
        elif (rand >= 0.5) and (rand < 0.75):
            return 2
        else:
            return 3
        
    def reset (self):
        self.current_state = self.starting_state
        return self.current_state
        
    def check_win (self):
        if self.current_state == self.win_state:
            return True
        return False

    # just reset for now...
    def close (self):
        self.current_state = self.starting_state
        return 1
    
    def step (self, action):
        # north
        if action == 0:
            next = (self.current_state[0] - 1, self.current_state[1])
        # south
        elif action == 1:
            next = (self.current_state[0] + 1, self.current_state[1])
        # east
        elif action == 2:
            next = (self.current_state[0], self.current_state[1] + 1)
        # west
        else:
            next = (self.current_state[0], self.current_state[1] - 1)

        terminate = False
        reward = 0
        # check if move is legal
        if (next[0] >= 0 and next[0] <= 9) and (next[1] >= 0 and next[1] <= 9):            
            illegal = 0
            for i in range(5):
                if next == (4, i):
                    illegal = 1
                    
            if (illegal == 0):
                self.current_state = next
                reward -= 0.01
            else:
                reward -= 1
        else:
            reward -= 1
        
        if self.check_win():
            reward += 100
            terminate = True
        
        return self.current_state, reward, terminate

### GridWorld Test

In [7]:
env = GridWorldv1(10, 10, (9, 0), (0, 0))
state = env.reset()
#action = agent.act(state)
action = env.sample_action()
state, reward, isDone = env.step(action)
print(state)
print(reward)
print(isDone)

(0, 0)
-1
False


### TPG Agent

In [8]:
def runAgent(args):
    agent = args[0]
    envName = args[1]
    scoreList = args[2]
    numEpisodes = args[3] # number of times to repeat game
    numFrames = args[4] 
    gen = args[5]
    
    # skip if task already done by agent
    if agent.taskDone(envName):
        print('Agent #' + str(agent.agentNum) + ' can skip.')
        scoreList.append((agent.team.id, agent.team.outcomes))
        return
    
    env = GridWorldv1(10, 10, (9, 0), (0, 0))
    valActs = 4
    
    limit = 100000000
    
    # create environment object from above
    scoreTotal = 0
    for ep in range(numEpisodes):
        state = env.reset()
        scoreEp = 0
        states = []
        step_map = np.zeros((10, 10))
        if numEpisodes > 1:
            #numRandFrames = random.randint(0,30)
            limitReached = 0
            for i in range(numFrames): # frame loop
#                 if i < numRandFrames:
#                     env.step(env.sample_action())
#                     continue
                    
                # action selection
                act = agent.act(state)
                
                state, reward, isDone = env.step(act)
                # for heatmap
                step_map[state] = step_map[state] + 1
                states.append(state)
                #print('State: ' + str(state))
                scoreEp += reward
                       
                if i > limit:
                    print('Limit Reached!')
                    limitReached = True
                
                # animate the winners to see what they are doing
                if isDone:
                    print('win!')
                    #animate_path(states)
                
                if isDone or limitReached:
                    break
                    
            print('Agent #' + str(agent.agentNum) + 
              ' | Ep #' + str(ep) + ' | Score: ' + str(scoreEp))
            
        scoreTotal += scoreEp
    scoreTotal /= numEpisodes
    env.close()
    agent.reward(scoreTotal, envName)
    scoreList.append((agent.team.id, agent.team.outcomes))

### All Together Now!

In [10]:
acts = 4
# pAtomic equal to 1 so teams are single nodes
    # starting off, teams don't reference teams
trainer = Trainer(actions=acts, teamPopSize=50, pActAtom=1.0, 
                  nRegisters=4, initMaxActProgSize=48, gap=0.2)

envName = 'GridWorldv1'
allScores = []
for gen in range(200):
    scoreList = []

    agents = trainer.getAgents()

    for agent in agents:
        runAgent([agent, envName, scoreList, 10, 1000, gen])

    # apply scores, must do this when multiprocessing
    # because agents can't refer to trainer
    teams = trainer.applyScores(scoreList)

    trainer.evolve(tasks=[envName]) # go into next gen

    # an easier way to track stats than the above example
    scoreStats = trainer.fitnessStats

    allScores.append((scoreStats['min'], scoreStats['max'], scoreStats['average']))

    print('Gen: ' + str(gen))
    print('Results so far: ' + str(allScores))

print('Results:\nMin, Max, Avg')
for score in allScores:
    print(score[0],score[1],score[2])

Agent #0 | Ep #0 | Score: -988.1199999999999
Agent #0 | Ep #1 | Score: -991.09
Agent #0 | Ep #2 | Score: -982.18
Agent #0 | Ep #3 | Score: -997.03
Agent #0 | Ep #4 | Score: -997.03
Agent #0 | Ep #5 | Score: -997.03
Agent #0 | Ep #6 | Score: -997.03
Agent #0 | Ep #7 | Score: -997.03
Agent #0 | Ep #8 | Score: -997.03
Agent #0 | Ep #9 | Score: -997.03
Agent #1 | Ep #0 | Score: -333.72999999999695
Agent #1 | Ep #1 | Score: -331.74999999999693
Agent #1 | Ep #2 | Score: -777.2500000000003
Agent #1 | Ep #3 | Score: -991.09
Agent #1 | Ep #4 | Score: -991.09
Agent #1 | Ep #5 | Score: -991.09
Agent #1 | Ep #6 | Score: -991.09
Agent #1 | Ep #7 | Score: -991.09
Agent #1 | Ep #8 | Score: -991.09
Agent #1 | Ep #9 | Score: -991.09
Agent #2 | Ep #0 | Score: -1000
Agent #2 | Ep #1 | Score: -1000
Agent #2 | Ep #2 | Score: -1000
Agent #2 | Ep #3 | Score: -1000
Agent #2 | Ep #4 | Score: -1000
Agent #2 | Ep #5 | Score: -1000
Agent #2 | Ep #6 | Score: -1000
Agent #2 | Ep #7 | Score: -1000
Agent #2 | Ep #8 |

Agent #24 | Ep #7 | Score: -1000
Agent #24 | Ep #8 | Score: -1000
Agent #24 | Ep #9 | Score: -1000
Agent #25 | Ep #0 | Score: -999.01
Agent #25 | Ep #1 | Score: -1000
Agent #25 | Ep #2 | Score: -1000
Agent #25 | Ep #3 | Score: -1000
Agent #25 | Ep #4 | Score: -1000
Agent #25 | Ep #5 | Score: -1000
Agent #25 | Ep #6 | Score: -1000
Agent #25 | Ep #7 | Score: -1000
Agent #25 | Ep #8 | Score: -1000
Agent #25 | Ep #9 | Score: -1000
Agent #26 | Ep #0 | Score: -991.09
Agent #26 | Ep #1 | Score: -991.09
Agent #26 | Ep #2 | Score: -991.09
Agent #26 | Ep #3 | Score: -991.09
Agent #26 | Ep #4 | Score: -991.09
Agent #26 | Ep #5 | Score: -991.09
Agent #26 | Ep #6 | Score: -991.09
Agent #26 | Ep #7 | Score: -991.09
Agent #26 | Ep #8 | Score: -991.09
Agent #26 | Ep #9 | Score: -991.09
Agent #27 | Ep #0 | Score: -995.05
Agent #27 | Ep #1 | Score: -997.03
Agent #27 | Ep #2 | Score: -997.03
Agent #27 | Ep #3 | Score: -997.03
Agent #27 | Ep #4 | Score: -997.03
Agent #27 | Ep #5 | Score: -997.03
Agent #27

Agent #48 | Ep #7 | Score: -1000
Agent #48 | Ep #8 | Score: -1000
Agent #48 | Ep #9 | Score: -1000
Agent #49 | Ep #0 | Score: -998.02
Agent #49 | Ep #1 | Score: -1000
Agent #49 | Ep #2 | Score: -1000
Agent #49 | Ep #3 | Score: -1000
Agent #49 | Ep #4 | Score: -1000
Agent #49 | Ep #5 | Score: -1000
Agent #49 | Ep #6 | Score: -1000
Agent #49 | Ep #7 | Score: -1000
Agent #49 | Ep #8 | Score: -1000
Agent #49 | Ep #9 | Score: -1000
Adding 7ca37c15-4261-4677-9b39-181850035c89 to trainer learners
Adding f47dc025-48b1-4e65-9b3f-a53ed3fecc17 to trainer learners
Adding 6e4b39f8-e5a2-434f-8fe3-bb48c0396878 to trainer learners
Adding 70804248-1e05-4bbb-9e68-ae290ce8713d to trainer learners
Adding 29029a90-dad5-4d03-8675-63865536eda9 to trainer learners
Adding 33b5f306-4897-4c32-9255-78348b2d360c to trainer learners
Adding 02686e7f-3aba-4063-b0e5-2455ca90be08 to trainer learners
Adding 2fd41b2e-e2af-4ac4-999c-f01afbe285a2 to trainer learners
Adding 84b8e3c1-5b5a-4f07-a565-4833f07147b9 to trainer le

Agent #43 | Ep #2 | Score: -1000
Agent #43 | Ep #3 | Score: -1000
Agent #43 | Ep #4 | Score: -1000
Agent #43 | Ep #5 | Score: -1000
Agent #43 | Ep #6 | Score: -1000
Agent #43 | Ep #7 | Score: -1000
Agent #43 | Ep #8 | Score: -1000
Agent #43 | Ep #9 | Score: -1000
Agent #44 | Ep #0 | Score: -999.01
Agent #44 | Ep #1 | Score: -1000
Agent #44 | Ep #2 | Score: -1000
Agent #44 | Ep #3 | Score: -1000
Agent #44 | Ep #4 | Score: -1000
Agent #44 | Ep #5 | Score: -1000
Agent #44 | Ep #6 | Score: -1000
Agent #44 | Ep #7 | Score: -1000
Agent #44 | Ep #8 | Score: -1000
Agent #44 | Ep #9 | Score: -1000
Agent #45 | Ep #0 | Score: -998.02
Agent #45 | Ep #1 | Score: -1000
Agent #45 | Ep #2 | Score: -1000
Agent #45 | Ep #3 | Score: -1000
Agent #45 | Ep #4 | Score: -1000
Agent #45 | Ep #5 | Score: -1000
Agent #45 | Ep #6 | Score: -1000
Agent #45 | Ep #7 | Score: -1000
Agent #45 | Ep #8 | Score: -1000
Agent #45 | Ep #9 | Score: -1000
Agent #46 | Ep #0 | Score: -997.03
Agent #46 | Ep #1 | Score: -997.03
Ag

Agent #40 | Ep #1 | Score: -991.09
Agent #40 | Ep #2 | Score: -991.09
Agent #40 | Ep #3 | Score: -991.09
Agent #40 | Ep #4 | Score: -991.09
Agent #40 | Ep #5 | Score: -991.09
Agent #40 | Ep #6 | Score: -991.09
Agent #40 | Ep #7 | Score: -991.09
Agent #40 | Ep #8 | Score: -991.09
Agent #40 | Ep #9 | Score: -991.09
Agent #41 | Ep #0 | Score: -997.03
Agent #41 | Ep #1 | Score: -997.03
Agent #41 | Ep #2 | Score: -997.03
Agent #41 | Ep #3 | Score: -997.03
Agent #41 | Ep #4 | Score: -997.03
Agent #41 | Ep #5 | Score: -997.03
Agent #41 | Ep #6 | Score: -997.03
Agent #41 | Ep #7 | Score: -997.03
Agent #41 | Ep #8 | Score: -997.03
Agent #41 | Ep #9 | Score: -997.03
Agent #42 | Ep #0 | Score: -1000
Agent #42 | Ep #1 | Score: -1000
Agent #42 | Ep #2 | Score: -1000
Agent #42 | Ep #3 | Score: -1000
Agent #42 | Ep #4 | Score: -1000
Agent #42 | Ep #5 | Score: -1000
Agent #42 | Ep #6 | Score: -1000
Agent #42 | Ep #7 | Score: -1000
Agent #42 | Ep #8 | Score: -1000
Agent #42 | Ep #9 | Score: -1000
Agent

...

Agent #40 | Ep #3 | Score: -991.09
Agent #40 | Ep #4 | Score: -991.09
Agent #40 | Ep #5 | Score: -991.09
Agent #40 | Ep #6 | Score: -991.09
Agent #40 | Ep #7 | Score: -991.09
Agent #40 | Ep #8 | Score: -991.09
Agent #40 | Ep #9 | Score: -991.09
Agent #41 | Ep #0 | Score: -9.999999999999831
Agent #41 | Ep #1 | Score: -9.999999999999831
Agent #41 | Ep #2 | Score: -9.999999999999831
Agent #41 | Ep #3 | Score: -9.999999999999831
Agent #41 | Ep #4 | Score: -9.999999999999831
Agent #41 | Ep #5 | Score: -9.999999999999831
Agent #41 | Ep #6 | Score: -9.999999999999831
Agent #41 | Ep #7 | Score: -9.999999999999831
Agent #41 | Ep #8 | Score: -9.999999999999831
Agent #41 | Ep #9 | Score: -9.999999999999831
Agent #42 | Ep #0 | Score: -1000
Agent #42 | Ep #1 | Score: -1000
Agent #42 | Ep #2 | Score: -1000
Agent #42 | Ep #3 | Score: -1000
Agent #42 | Ep #4 | Score: -1000
Agent #42 | Ep #5 | Score: -1000
Agent #42 | Ep #6 | Score: -1000
Agent #42 | Ep #7 | Score: -1000
Agent #42 | Ep #8 | Score: -100

Agent #40 | Ep #4 | Score: -10.98999999999981
Agent #40 | Ep #5 | Score: -10.98999999999981
Agent #40 | Ep #6 | Score: -10.98999999999981
Agent #40 | Ep #7 | Score: -10.98999999999981
Agent #40 | Ep #8 | Score: -10.98999999999981
Agent #40 | Ep #9 | Score: -10.98999999999981
Agent #41 | Ep #0 | Score: -991.09
Agent #41 | Ep #1 | Score: -991.09
Agent #41 | Ep #2 | Score: -991.09
Agent #41 | Ep #3 | Score: -991.09
Agent #41 | Ep #4 | Score: -991.09
Agent #41 | Ep #5 | Score: -991.09
Agent #41 | Ep #6 | Score: -991.09
Agent #41 | Ep #7 | Score: -991.09
Agent #41 | Ep #8 | Score: -991.09
Agent #41 | Ep #9 | Score: -991.09
Agent #42 | Ep #0 | Score: -991.09
Agent #42 | Ep #1 | Score: -991.09
Agent #42 | Ep #2 | Score: -991.09
Agent #42 | Ep #3 | Score: -991.09
Agent #42 | Ep #4 | Score: -991.09
Agent #42 | Ep #5 | Score: -991.09
Agent #42 | Ep #6 | Score: -991.09
Agent #42 | Ep #7 | Score: -991.09
Agent #42 | Ep #8 | Score: -991.09
Agent #42 | Ep #9 | Score: -991.09
Agent #43 | Ep #0 | Scor

Agent #40 | Ep #1 | Score: -1000
Agent #40 | Ep #2 | Score: -1000
Agent #40 | Ep #3 | Score: -1000
Agent #40 | Ep #4 | Score: -1000
Agent #40 | Ep #5 | Score: -1000
Agent #40 | Ep #6 | Score: -1000
Agent #40 | Ep #7 | Score: -1000
Agent #40 | Ep #8 | Score: -1000
Agent #40 | Ep #9 | Score: -1000
Agent #41 | Ep #0 | Score: -991.09
Agent #41 | Ep #1 | Score: -991.09
Agent #41 | Ep #2 | Score: -991.09
Agent #41 | Ep #3 | Score: -991.09
Agent #41 | Ep #4 | Score: -991.09
Agent #41 | Ep #5 | Score: -991.09
Agent #41 | Ep #6 | Score: -991.09
Agent #41 | Ep #7 | Score: -991.09
Agent #41 | Ep #8 | Score: -991.09
Agent #41 | Ep #9 | Score: -991.09
Agent #42 | Ep #0 | Score: -997.03
Agent #42 | Ep #1 | Score: -997.03
Agent #42 | Ep #2 | Score: -997.03
Agent #42 | Ep #3 | Score: -997.03
Agent #42 | Ep #4 | Score: -997.03
Agent #42 | Ep #5 | Score: -997.03
Agent #42 | Ep #6 | Score: -997.03
Agent #42 | Ep #7 | Score: -997.03
Agent #42 | Ep #8 | Score: -997.03
Agent #42 | Ep #9 | Score: -997.03
Age

Agent #40 | Ep #3 | Score: -997.03
Agent #40 | Ep #4 | Score: -997.03
Agent #40 | Ep #5 | Score: -997.03
Agent #40 | Ep #6 | Score: -997.03
Agent #40 | Ep #7 | Score: -997.03
Agent #40 | Ep #8 | Score: -997.03
Agent #40 | Ep #9 | Score: -997.03
Agent #41 | Ep #0 | Score: -102.07000000000431
Agent #41 | Ep #1 | Score: -9.999999999999831
Agent #41 | Ep #2 | Score: -9.999999999999831
Agent #41 | Ep #3 | Score: -9.999999999999831
Agent #41 | Ep #4 | Score: -9.999999999999831
Agent #41 | Ep #5 | Score: -9.999999999999831
Agent #41 | Ep #6 | Score: -9.999999999999831
Agent #41 | Ep #7 | Score: -9.999999999999831
Agent #41 | Ep #8 | Score: -9.999999999999831
Agent #41 | Ep #9 | Score: -9.999999999999831
Agent #42 | Ep #0 | Score: -9.999999999999831
Agent #42 | Ep #1 | Score: -9.999999999999831
Agent #42 | Ep #2 | Score: -9.999999999999831
Agent #42 | Ep #3 | Score: -9.999999999999831
Agent #42 | Ep #4 | Score: -9.999999999999831
Agent #42 | Ep #5 | Score: -9.999999999999831
Agent #42 | Ep #6 

Agent #40 | Ep #0 | Score: -9.999999999999831
Agent #40 | Ep #1 | Score: -9.999999999999831
Agent #40 | Ep #2 | Score: -9.999999999999831
Agent #40 | Ep #3 | Score: -9.999999999999831
Agent #40 | Ep #4 | Score: -9.999999999999831
Agent #40 | Ep #5 | Score: -9.999999999999831
Agent #40 | Ep #6 | Score: -9.999999999999831
Agent #40 | Ep #7 | Score: -9.999999999999831
Agent #40 | Ep #8 | Score: -9.999999999999831
Agent #40 | Ep #9 | Score: -9.999999999999831
Agent #41 | Ep #0 | Score: -991.09
Agent #41 | Ep #1 | Score: -991.09
Agent #41 | Ep #2 | Score: -991.09
Agent #41 | Ep #3 | Score: -991.09
Agent #41 | Ep #4 | Score: -991.09
Agent #41 | Ep #5 | Score: -991.09
Agent #41 | Ep #6 | Score: -991.09
Agent #41 | Ep #7 | Score: -991.09
Agent #41 | Ep #8 | Score: -991.09
Agent #41 | Ep #9 | Score: -991.09
Agent #42 | Ep #0 | Score: -1000
Agent #42 | Ep #1 | Score: -1000
Agent #42 | Ep #2 | Score: -1000
Agent #42 | Ep #3 | Score: -1000
Agent #42 | Ep #4 | Score: -1000
Agent #42 | Ep #5 | Score